# Fake News Detector
Building a system to identify unreliable news articles.

[Kaggle compitition submession](https://www.kaggle.com/aalmokainzi/fake-news-detector) <br>
[Data source](https://www.kaggle.com/c/fake-news/data) <br>
[Code source](https://data-flair.training/blogs/advanced-python-project-detecting-fake-news/)

In [137]:
# Make necessary imports
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [138]:
# Read the data
df = pd.read_csv('data/train.csv')

In [139]:
# Rows and columns in the data
df.shape

(20800, 5)

In [140]:
# Take a glimpse of the data
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [141]:
# Get more information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [142]:
# Check for missing data in each feature/column
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [143]:
# Drop unrelated features first, then drop missing data
df = df.drop(columns=['title', 'author']).dropna()

In [144]:
# Double check missing data
df.isna().sum()

id       0
text     0
label    0
dtype: int64

In [145]:
# Get the labels. 1: unreliable, 0: reliable
labels = df.label
labels.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [146]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['text'], labels, test_size = 0.2, random_state = 7)

In [147]:
# Initialize a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', max_df = 0.7)

# Fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [148]:
# Initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter = 50)
pac.fit(tfidf_train, y_train)

# Predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100, 2)}%')

Accuracy: 95.98%


In [149]:
# Build confusion matrix. 1: unreliable, 0: reliable
confusion_matrix(y_test, y_pred, labels=[1, 0])

array([[2031,   95],
       [  72, 1955]])

In [150]:
# Test on a random news headline or text
pac.predict(tfidf_vectorizer.transform(['WATCH LIVE: Rudy Giuliani details findings of Trump legal team investigations, voter fraud allegations']))

array([1])

<hr>

## Test Data

In [151]:
# Read the data
test_data = pd.read_csv('data/test.csv')

In [152]:
# Assign ids to an object to use it later for Kaggle submission
test_id = test_data['id']

In [153]:
# Rows and columns in the data
test_data.shape

(5200, 4)

In [154]:
# Take a glimpse of the data
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [155]:
# Get some information about the data
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


In [156]:
# How many missing data in each feature/column
test_data.isna().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [157]:
# Drop unrelated features first, then fill missing data
# Fill NAs instead of dropping, since the submission is expecting same number of observations as the original one
test_data = test_data.drop(columns=['id','title', 'author']).fillna('fake and unreliable')

In [158]:
# Double check missing data
test_data.isna().sum()

text    0
dtype: int64

In [159]:
# Confirm that cleaned test data has same observations as the original one (i.e. 5200)
test_data.shape

(5200, 1)

In [160]:
# Transform test data
test_vectorized = tfidf_vectorizer.transform(test_data['text'])

In [161]:
# Predict test data
test_pred = pac.predict(test_vectorized)

In [162]:
# Join test data's ids with their respective predicted labels
predictions = pd.DataFrame({'id':test_id, 'label':test_pred})
predictions.shape

(5200, 2)

In [163]:
predictions.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1
